In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [3]:
img = cv2.imread('sIMG_8253.JPG')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

cv2.imshow("img", img)
cv2.imshow("gray", gray)
cv2.imshow("ret", thresh)
cv2.imshow("thresh", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
img = cv2.imread('sIMG_8253.JPG')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)


# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)


# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)


# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
result_dist_transform = cv2.normalize(dist_transform, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8UC1)
ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(),255, cv2.THRESH_BINARY)



sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)



ret, markers = cv2.connectedComponents(sure_fg)


markers = markers+1


markers[unknown==255] = 0


markers = cv2.watershed(img, markers)

img[markers == -1] = [255, 0, 0]
img[markers == 1] = [255, 255, 0]

cv2.imshow("dist_transform", result_dist_transform)
cv2.imshow("unknown", unknown)
cv2.imshow("sure_fg", sure_fg)
cv2.imshow("sure_bg", sure_bg)
cv2.imshow("result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()